In [1]:
import sys
sys.path.append("../scripts/")
from cleaner import DataCleaner
from assistants import assistant
from text_preprocessor import Processor

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.filterwarnings('ignore')

In [4]:
assistant = assistant()
cleaner = DataCleaner()
processor = Processor()

In [9]:
repo="https://github.com/michaelgetachew-abebe/Prompt-Engineering-LLMs.git"
news_dataframe=assistant.read_from_dvc("data/cleaned_news.csv",repo,"60787a4c729bc276d60d03c0c0823a974cf1c93e",low_memory=False)
news_dataframe=news_dataframe.sample(frac=1)

In [11]:
train_news=news_dataframe.head(7)
test_news=news_dataframe.tail(3)